In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tifffile as tf
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.integrate import odeint
import time
from numpy import linalg as LA
import random
from itertools import cycle

%matplotlib inline

In [ ]:
def nanRobustBlur(I, dim):
    V=I.copy()
    V[I!=I]=0
    VV=cv2.blur(V,dim)   
    W=0*I.copy()+1
    W[I!=I]=0
    WW=cv2.blur(W,dim)    
    Z=VV/WW
    return Z 

In [ ]:
# anisotropy = np.random.rand(40, 40)
# orientation = np.zeros((40, 40))

anisotropy = np.zeros((40, 40)) + 1
orientation = np.zeros((40, 40))

In [ ]:
plt.imshow(anisotropy)

In [ ]:
def return_smooth(orientation, anisotropy):
    U, V =  anisotropy*np.cos(2 * orientation), anisotropy*np.sin(2 * orientation)
    USmooth = nanRobustBlur(U, (5, 5)) # plot smoothed vector field
    VSmooth = nanRobustBlur(V, (5, 5)) # plot smoothed vector field
    azimuthSmooth = (0.5*np.arctan2(VSmooth,USmooth)) % np.pi
    RSmooth = np.sqrt(USmooth**2+VSmooth**2)
    
    return RSmooth, azimuthSmooth

In [ ]:
anisotropy, orientation = return_smooth(orientation, anisotropy)

In [ ]:
def return_D(position):
    scale_value = anisotropy[position[0]][position[1]]
    theta = orientation[position[0]][position[1]]
    
    if abs(scale_value) > 1:
        scale_matrix = np.matrix([[scale_value, 0], [0, 1]])
    else:
        scale_matrix = np.matrix([[1, 0], [0, scale_value]])
    
    angle_matrix = np.matrix([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    angle_matrix_2 = np.matrix([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
    
    D_matrix = np.matmul(np.matmul(angle_matrix, scale_matrix), angle_matrix_2)
    
    return D_matrix

In [ ]:
memory_inverse = {}

In [ ]:
D1 = np.zeros_like(anisotropy, dtype=np.float32)
D2 = np.zeros_like(anisotropy, dtype=np.float32)
D3 = np.zeros_like(anisotropy, dtype=np.float32)
D4 = np.zeros_like(anisotropy, dtype=np.float32)

x = np.arange(0, anisotropy.shape[0], 1)
y = np.arange(0, anisotropy.shape[1], 1)

for i in x:
    for j in y:
        scale_matrix = return_D([i, j])
        memory_inverse[(i, j)] = np.linalg.inv(scale_matrix)

        D1[i, j] = scale_matrix[0, 0]
        D2[i, j] = scale_matrix[0, 1]
        D3[i, j] = scale_matrix[1, 0]
        D4[i, j] = scale_matrix[1, 1]

In [ ]:
from scipy import interpolate

In [ ]:
plt.imshow(D1)

In [ ]:
D1_interp = interpolate.interp2d(x, y, D1.T, kind='cubic')
D2_interp = interpolate.interp2d(x, y, D2.T, kind='cubic')
D3_interp = interpolate.interp2d(x, y, D3.T, kind='cubic')
D4_interp = interpolate.interp2d(x, y, D4.T, kind='cubic')

In [ ]:
def return_D_interp_list(positions):
    
    matrices = []
    
    for position in positions:
        D1_value = D1_interp(position[0], position[1])[0]
        D2_value = D2_interp(position[0], position[1])[0]
        D3_value = D3_interp(position[0], position[1])[0]
        D4_value = D4_interp(position[0], position[1])[0]
        
        matrices.append([[D1_value, D2_value], [D3_value, D4_value]])
            
    return np.stack(matrices, axis=0)

In [ ]:
def return_D_interp(position):
    D1_value = D1_interp(position[0], position[1])[0]
    D2_value = D2_interp(position[0], position[1])[0]
    D3_value = D3_interp(position[0], position[1])[0]
    D4_value = D4_interp(position[0], position[1])[0]
    
    return np.matrix([[D1_value, D2_value], [D3_value, D4_value]])

In [ ]:
D1_int = np.zeros_like(anisotropy, dtype=np.float32)
D2_int = np.zeros_like(anisotropy, dtype=np.float32)
D3_int = np.zeros_like(anisotropy, dtype=np.float32)
D4_int = np.zeros_like(anisotropy, dtype=np.float32)

for i in x:
    for j in y:
        scale_matrix = return_D_interp([i, j])

        D1_int[i, j] = scale_matrix[0, 0]
        D2_int[i, j] = scale_matrix[0, 1]
        D3_int[i, j] = scale_matrix[1, 0]
        D4_int[i, j] = scale_matrix[1, 1]

In [ ]:
plt.imshow(D1_int)

In [ ]:
gamma = 0.5
def force_function(r):
    if 0 < r < 1:
        return r - 1
    elif 1 <= r <= 1 + gamma:
        return ((r-1)*(1+gamma-r)**2)/gamma**2
    elif r > 1 + gamma:
        return 0

In [ ]:
gamma = 0.5
def force_function_list(r):
    
    mask_1 = np.logical_and(r < 1, r > 0)
    mask_2 = np.logical_and(r >= 1, r <= 1 + gamma)
    mask_3 = r > 1+gamma
    
    return mask_1*(r-1) + mask_2*(((r-1)*(1+gamma-r)**2)/gamma**2) + mask_3*(0)

In [ ]:
def return_D_inverse(pos_a, pos_b, approx=True):
    if approx:
        pos_a = ([int(x) for x in pos_a])
        pos_b = ([int(x) for x in pos_b])
        pos_a = (pos_a[0], pos_a[1])
        pos_b = (pos_b[0], pos_b[1])
        
        if pos_a not in memory_inverse.keys() and pos_b not in memory_inverse.keys():
            D_inverse_ab = np.identity(2)
        elif pos_a not in memory_inverse.keys():
            D_inverse_ab = memory_inverse[pos_b]
        elif pos_b not in memory_inverse.keys():
            D_inverse_ab = memory_inverse[pos_a]
        else:
            D_inverse_ab = (memory_inverse[pos_a] + memory_inverse[pos_b])/2.0

        return D_inverse_ab
    else:
        mid_pos = tuple(np.add(pos_a, pos_b)/2.0)
        D_ab = return_D_interp(mid_pos)
        
        return np.linalg.inv(D_ab)

In [ ]:
def return_D_inverse_list(pos_a, pos_b_positions, approx=True):
    inverses = []
    if approx:
        for pos_b in pos_b_positions:
            inverses.append(return_D_inverse(pos_a, pos_b, approx))
            
        return inverses
    else:
        mid_positions = np.add(pos_a, pos_b_positions)/2.0      
        D_ab_list = return_D_interp_list(mid_positions)
        identity = np.identity(D_ab_list[0].shape[1], dtype=D_ab_list[0].dtype)
        
        return np.array([np.linalg.solve(x, identity) for x in D_ab_list])

In [ ]:
alpha = 1000.0
def return_x_ab(pos_a, pos_b, approx=True):
    D_inv_ab = return_D_inverse(pos_a, pos_b, approx)
    
    pos_a = np.asarray(pos_a, dtype='float32')
    pos_b = np.asarray(pos_b, dtype='float32')
    y_ab = pos_a - pos_b
    y_ab = y_ab.reshape((2, 1))
    
    return D_inv_ab.dot(y_ab)/(2*alpha), D_inv_ab

In [ ]:
def return_x_ab_list(pos_a, pos_b_positions, approx=True):
    
    D_inv_ab_stack = return_D_inverse_list(pos_a, pos_b_positions, approx)
    pos_a = np.asarray(pos_a, dtype='float32')
    pos_b = np.asarray(pos_b_positions, dtype='float32')
    
    y_ab_stack = pos_a - pos_b_positions
    y_ab_stack = y_ab_stack.reshape(len(pos_b_positions), 2, 1)
    
    return np.diagonal(np.dot(D_inv_ab_stack, y_ab_stack)[:,:,:,0], axis1=0, axis2=2)/(2*alpha), D_inv_ab_stack

In [ ]:
def return_force(pos_a, pos_b, approx=True):
    
    print(memory_force)
    
    if (pos_a, pos_b) in memory_force.keys():
        return memory_force[(pos_a, pos_b)]
    elif (pos_b, pos_a) in memory_force.keys():
        return -memory_force[(pos_b, pos_a)]
    else:
        print("Hello")
        x_ab, D_inv_ab = return_x_ab(pos_a, pos_b, approx)
        x_ab_dist = LA.norm(x_ab)        
        memory_force[(pos_a, pos_b)] = (-force_function(x_ab_dist)*D_inv_ab.dot(x_ab))/(2*alpha*x_ab_dist)
        
        return memory_force[(pos_a, pos_b)]

In [ ]:
def return_total_force(pos_a, pos_b_positions, approx=True):
    
    total_force = np.zeros((2, 1))
    not_in_memory_pos = []
    
    for pos_b in pos_b_positions:
        if (pos_a, pos_b) in memory_force.keys():
            total_force += memory_force[(pos_a, pos_b)]
        elif (pos_b, pos_a) in memory_force.keys():
            total_force += -memory_force[(pos_b, pos_a)]
        else:
            not_in_memory_pos.append(pos_b)

    x_ab_list, D_inv_ab_stack = return_x_ab_list(pos_a, not_in_memory_pos, approx)
    x_ab_dist_list = LA.norm(x_ab_list, axis=0)
    
    mult = np.transpose(x_ab_list).reshape(x_ab_list.shape[1], x_ab_list.shape[0], 1)
    
    force_list = force_function_list(x_ab_dist_list)*np.diagonal(np.dot(D_inv_ab_stack, mult)[:,:,:,0], axis1=0, axis2=2)/(2*alpha*x_ab_dist_list)

    total_force += np.sum(force_list, axis = 1).reshape((2, 1))
    return total_force

In [ ]:
def return_total_force_alter(pos_a, pos_b_positions, approx=True):
    total_force = np.zeros((2, 1))
    not_in_memory_pos = []
    D_inverse_list = []
    
    identity = np.identity(2, dtype=float)
    
    for pos_b in pos_b_positions:
        if (pos_a, pos_b) in memory_force.keys():
            total_force += memory_force[(pos_a, pos_b)]
        elif (pos_b, pos_a) in memory_force.keys():
            total_force += -memory_force[(pos_b, pos_a)]
        else:
            if pos_b != pos_a:
                not_in_memory_pos.append(pos_b)

                if approx:
                    D_inverse_list.append(return_D_inverse(pos_a, pos_b, approx))
                else:
                    position = tuple(np.add(pos_a, pos_b)/2.0)

                    D1_value = D1_interp(position[0], position[1])[0]
                    D2_value = D2_interp(position[0], position[1])[0]
                    D3_value = D3_interp(position[0], position[1])[0]
                    D4_value = D4_interp(position[0], position[1])[0]

                    matrix = np.array([[D1_value, D2_value], [D3_value, D4_value]])

                    D_inverse_list.append(np.linalg.solve(matrix, identity))
    
    if len(not_in_memory_pos) > 0:
        D_inverse_list = np.array(D_inverse_list)

        pos_a_np = np.asarray(pos_a, dtype='float32')

        y_ab_stack = pos_a_np - not_in_memory_pos
        y_ab_stack = y_ab_stack.reshape(len(not_in_memory_pos), 2, 1)

        x_ab_list = np.transpose(np.einsum('ijk,ikl->ijl', D_inverse_list, y_ab_stack)[:,:,0])/(2*alpha)
        x_ab_dist_list = LA.norm(x_ab_list, axis=0)

        mult = np.transpose(x_ab_list).reshape(x_ab_list.shape[1], x_ab_list.shape[0], 1)
        force_list = -force_function_list(x_ab_dist_list)*np.transpose(np.einsum('ijk,ikl->ijl', D_inverse_list, mult)[:,:,0])/(2*alpha*x_ab_dist_list)
        missed_points = list(zip(cycle([pos_a]), not_in_memory_pos))
        memory_force.update(zip(missed_points, list(np.transpose(force_list).reshape((len(not_in_memory_pos), 2, 1)))))

        total_force += np.sum(force_list, axis = 1).reshape((2, 1))   
            
    return total_force

In [ ]:
# memory_force = {}

# pos_a = (1.0, 1.0)
# pos_b_positions = []

# num_points = 100

# for i in range(num_points):
#     pos_b_positions.append((random.uniform(0, 40), random.uniform(0, 40)))

# start = time.time()
# tf = np.zeros((2, 1))

# for i in range(num_points):
#     tf += return_force(pos_a, pos_b_positions[i], approx=False)

# print(tf)
# end = time.time()
# print(end - start)

In [ ]:
# memory_force = {}

# import time

# start = time.time()
# print(return_total_force_alter(pos_a, pos_b_positions, approx=False))
# end = time.time()
# print(end - start)

In [ ]:
initial_num_points = 800
final_points = 500
random_rows = np.random.choice(anisotropy.shape[0], initial_num_points)
random_columns = np.random.choice(anisotropy.shape[1], initial_num_points)
random_points = list(zip(random_rows, random_columns))

In [ ]:
det_D_list = [np.linalg.det(return_D_interp(pos)) for pos in random_points]
max_det = max(det_D_list)
det_D_list = [x/max_det for x in det_D_list]
points_rejected = np.random.choice(len(random_points), initial_num_points - final_points, det_D_list)
final_points = [random_points[i] for i in range(len(random_points)) if i not in points_rejected]

In [ ]:
eig_D = [max(np.linalg.eig(return_D_interp(pos))[0]) for pos in random_points]
max_eig = max(eig_D)

In [ ]:
edge_length = 2*alpha*max_eig*(1+gamma)

def bin_coords(pos):
    x_coord = np.floor(pos[0]/edge_length)
    y_coord = np.floor(pos[1]/edge_length)
    
    return (x_coord, y_coord)

In [ ]:
curr_points = final_points

def form_dict(points):
    dict_points = {}

    for point in points:
        bin_coord = bin_coords(point)

        if bin_coord in dict_points.keys():
            dict_points[bin_coord].append(point)
        else:
            dict_points[bin_coord] = [point]
    
    return dict_points

In [ ]:
def total_force_on_point(index, list_points, map_points, approx=True):
    p_a = list_points[index]
    bin_coord = bin_coords(p_a)
    bin_x = bin_coord[0]
    bin_y = bin_coord[1]

    points_cons = []

    points_cons += map_points[bin_coord]

    if bin_x - 1 >= 0:
        if (bin_x - 1, bin_y) in map_points.keys():
            points_cons += map_points[(bin_x - 1, bin_y)]

    if bin_y - 1 >= 0:
        if (bin_x, bin_y - 1) in map_points.keys():
            points_cons += map_points[(bin_x, bin_y - 1)]

    if bin_x + 1 < anisotropy.shape[0]:
        if (bin_x + 1, bin_y) in map_points.keys():
            points_cons += map_points[(bin_x + 1, bin_y)]

    if bin_y + 1 < anisotropy.shape[1]:
        if (bin_x, bin_y + 1) in map_points.keys():
            points_cons += map_points[(bin_x, bin_y + 1)]
    
    total_force = return_total_force_alter(p_a, points_cons, approx)
    
    total_force_alter = np.zeros((2, 1))
    
    for point in points_cons:
        if point != p_a:
            f = return_force(p_a, point, approx)
            total_force_alter += f
            
    diff = LA.norm(total_force_alter - total_force)

    return total_force_alter, len(points_cons), diff

In [ ]:
def diff_equation(y, t, c_drag, total_force):
    p0, p1, r0, r1 = y
    dydt = [r0, r1, total_force[0] - c_drag*r0, total_force[1] - c_drag*r1]
    return dydt

def solve_particle_path(index, initial_pos, map_points, time_integrate, approx=True):
    c_drag = 1.0
    y0 = [initial_pos[index][0], initial_pos[index][1], 0.0, 0.0]
    
    t = np.linspace(0, time_integrate, 10)
    
    tf, tot_points, diff = total_force_on_point(index, initial_pos, map_points, approx)
    
    start_int = time.clock()
    sol = odeint(diff_equation, y0, t, args=(c_drag, tf))
    
    return sol, tot_points, diff

In [ ]:
num_iterations = 40
map_points = form_dict(curr_points)
max_time = 5.0
min_time = 1.0

slope = (max_time - min_time)/(1 - num_iterations)
constant = (num_iterations*max_time - min_time)/(num_iterations - 1)

start_algo = time.clock()
for k in range(num_iterations):
    start_iteration = time.clock()
    final_positions = []
    num_particles = 0
    total_dist = 0
    new_map_points = {}
    avg_total_points = 0
    memory_force = {}
    total_diff = 0

    for i in range(len(curr_points)):
        time_integrate = slope*k + constant
        
        if k < -1:
            sol, tot_points, diff = solve_particle_path(i, curr_points, map_points, time_integrate, approx=True)
        else:
            sol, tot_points, diff = solve_particle_path(i, curr_points, map_points, time_integrate, approx=False)
        
        total_diff += diff
        plt.plot(sol[:,0], sol[:,1])
        plt.scatter(sol[-1, 0], sol[-1, 1], s=5)
        
        if sol[-1, 0] >= -10 and sol[-1, 0] <= anisotropy.shape[0] + 10 and \
           sol[-1, 1] >= -10 and sol[-1, 1] <= anisotropy.shape[1] + 10:
            final_positions.append((sol[-1, 0], sol[-1, 1]))
            num_particles += 1
            avg_total_points += tot_points
            total_dist += LA.norm(np.array([sol[-1, 0], sol[-1, 1]]) - np.array([sol[0, 0], sol[0, 1]]))     

            new_bin_coord = bin_coords((sol[-1, 0], sol[-1, 1]))

            if new_bin_coord in new_map_points.keys():
                new_map_points[new_bin_coord].append((sol[-1, 0], sol[-1, 1]))
            else:
                new_map_points[new_bin_coord] = [(sol[-1, 0], sol[-1, 1])]
    
    if k%4 == 0: 
        print("Distances moved by particle:", total_dist/num_particles)
        print("Number of particles:", num_particles)
        print("Number of particles for force:", avg_total_points/num_particles)
        end_iteration = time.clock()
        print("Time taken for iteration: ", end_iteration - start_iteration)
        print("Total diff: ", total_diff)
        print("")
    
    curr_points = final_positions
    map_points = new_map_points

end_algo = time.clock()

print("Total time taken: ", end_algo - start_algo)

In [ ]:
final_positions = np.array(final_positions)
final_positions = np.around(final_positions, decimals=0)

In [ ]:
for i in range(len(final_positions)):
    if final_positions[i][0] >= 0 and final_positions[i][0] <= anisotropy.shape[0] and final_positions[i][1] >= 0 and final_positions[i][1] <= anisotropy.shape[1]:
        plt.scatter(final_positions[i][0], final_positions[i][1], s=20)

In [ ]:
USmooth, VSmooth = anisotropy*np.cos(orientation), anisotropy*np.sin(orientation)
final_positions = final_positions.astype(int)

In [ ]:
Plotting_X = []
Plotting_Y = []
Plotting_U = []
Plotting_V = []
Plotting_orient = []

USmooth = USmooth*0.1
VSmooth = VSmooth*0.1

for i in range(len(final_positions)):
    if final_positions[i][0] >= 0 and final_positions[i][0] < anisotropy.shape[0] and final_positions[i][1] >= 0 and final_positions[i][1] < anisotropy.shape[1]:
        Plotting_X.append(final_positions[i][0])
        Plotting_Y.append(final_positions[i][1])
        Plotting_U.append(USmooth[final_positions[i][0], final_positions[i][1]])
        Plotting_V.append(VSmooth[final_positions[i][0], final_positions[i][1]])
        Plotting_orient.append(orientation[final_positions[i][0], final_positions[i][1]])     

Plotting_color = Plotting_orient

In [ ]:
cmapImage = 'gray'
cmapOrient = 'hsv'
linecolor = 'g'
alpha = 0.6
linewidth = 0.5

im_ax = plt.imshow(anisotropy, cmap=cmapImage)
plt.title('Orientation')
plt.quiver(Plotting_Y, Plotting_X,
           Plotting_U, Plotting_V, Plotting_color,
           cmap=cmapOrient,
           edgecolor=linecolor, facecolor=linecolor, units='xy', alpha=alpha, width=linewidth,
           headwidth=0, headlength=0, headaxislength=0,
           scale_units = 'xy', scale = 1, angles = 'uv', pivot = 'mid')

In [ ]:
nY, nX = anisotropy.shape
Y, X = np.mgrid[0:nY,0:nX] # notice the reversed order of X and Y
spacing = 2
    
# Sample variables at the specified spacing.
Plotting_X = X[::spacing, ::spacing]
Plotting_Y = Y[::spacing, ::spacing]
Plotting_U = USmooth[::spacing, ::spacing]
Plotting_V = VSmooth[::spacing, ::spacing]

threshold = np.ones_like(X) # no threshold
Plotting_thres = threshold[::spacing, ::spacing]
Plotting_orient=orientation[::spacing, ::spacing]

thresholdIdx = Plotting_thres==1
Plotting_X=Plotting_X[thresholdIdx]
Plotting_Y=Plotting_Y[thresholdIdx]
Plotting_U=Plotting_U[thresholdIdx]
Plotting_V=Plotting_V[thresholdIdx]
Plotting_orient=Plotting_orient[thresholdIdx]
Plotting_color=Plotting_orient

In [ ]:
im_ax = plt.imshow(anisotropy, cmap=cmapImage)
plt.title('Orientation')
plt.quiver(Plotting_X, Plotting_Y,
           Plotting_U, Plotting_V, Plotting_color,
           cmap=cmapOrient,
           edgecolor=linecolor, facecolor=linecolor, units='xy', alpha=alpha, width=linewidth,
           headwidth=0, headlength=0, headaxislength=0,
           scale_units = 'xy', scale = 1, angles = 'uv', pivot = 'mid')

In [ ]:
USmooth, VSmooth = anisotropy*np.cos(orientation), anisotropy*np.sin(orientation)

Plotting_X = []
Plotting_Y = []
Plotting_U = []
Plotting_V = []
Plotting_orient = []

USmooth = USmooth*0.1
VSmooth = VSmooth*0.1

for i in range(len(final_points)):
        Plotting_X.append(final_points[i][0])
        Plotting_Y.append(final_points[i][1])
        Plotting_U.append(USmooth[final_points[i][0], final_points[i][1]])
        Plotting_V.append(VSmooth[final_points[i][0], final_points[i][1]])
        Plotting_orient.append(orientation[final_points[i][0], final_points[i][1]])     

Plotting_color = Plotting_orient

In [ ]:
cmapImage = 'gray'
cmapOrient = 'hsv'
linecolor = 'g'
alpha = 0.6
linewidth = 0.5

im_ax = plt.imshow(anisotropy, cmap=cmapImage)
plt.title('Orientation')
plt.quiver(Plotting_Y, Plotting_X,
           Plotting_U, Plotting_V, Plotting_color,
           cmap=cmapOrient,
           edgecolor=linecolor, facecolor=linecolor, units='xy', alpha=alpha, width=linewidth,
           headwidth=0, headlength=0, headaxislength=0,
           scale_units = 'xy', scale = 1, angles = 'uv', pivot = 'mid')

In [ ]:
A = np.array([[1,2],[3,4]])
x = np.array([[1, 2]]).reshape((2, 1))

B = np.array([[1,3],[3,5]])
y = np.array([[1, 3]]).reshape((2, 1))

In [ ]:
A.dot(x)

In [ ]:
C = np.stack((A, B))

In [ ]:
Z = np.stack((x, y))

In [ ]:
Z.shape

In [ ]:
np.dot(C, Z)[np.arange(2),:,np.arange(2),0]

In [ ]:
np.diagonal(np.dot(C, Z)[:,:,:,0], axis1=0, axis2=2)

In [ ]:
(C @ Z)[:,:,0]

In [ ]:
np.transpose(np.einsum('ijk,ikl->ijl', C, Z)[:,:,0])